## Итераторы, итерируемые объекты

**1.** (2) Реализуйте класс `BinTree` двоичного дерева, итерирование по которму происходит в порядке обхода в глубину.

In [9]:
class Node:
    def __init__(self, left=None, right=None, data=None):
        self.left = left
        self.right = right
        self.data = data

    def __str__(self):
        return str(self.data)

class Tree:
    
    def update_tree(self):
        self.dfs_iter = iter(self.InOrder())
    
    def __init__(self, expression):
        self.root = None
        self.add(expression)
        self.update_tree()
    
    def add(self, expression):
        token_list = list(''.join(expression))
        self.root = self._addToTree(token_list)
        self.update_tree()

    def _addToTree(self, token_list):
        if token_list:
            brackets = 0
            index_sym = 0
            for inx, sym in enumerate(token_list):
                if sym == '(': brackets += 1
                if sym == ')': brackets -= 1
                if (sym != '(') and (sym != ')'): index_sym = inx
                if (brackets == 0) and (sym != ')') and (sym != '('):
                    index_sym = inx
                    break
            return Node(self._addToTree(token_list[1:index_sym]),
                        self._addToTree(token_list[index_sym+2:-1]),
                        token_list[index_sym])

    def InOrder(self):
        return self._InOrder(self.root)

    def _InOrder(self, node):
        res = []
        if node:
            res = self._InOrder(node.left)
            res.append(node.data)
            res = res + self._InOrder(node.right)
        return res
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return next(self.dfs_iter)


tree = Tree('((c)b(d))a(((g)f(h))e)')


print('Обратный обход')
for i in tree:
    print(i)

print(tree.InOrder())

Обратный обход
c
b
d
a
g
f
h
e
['c', 'b', 'd', 'a', 'g', 'f', 'h', 'e']


**2.** (3) Скачайте архив, и положите его в папку с неделей 9. 

Создаейте класс `TextLoader`, который принимает в инициализаторе адрес архива. 

Затем добейтесь от класса следующего поведения:
- При инициализации объекта (т.е. в методе `__init__`) этот класс должен разархивировать архив в какую-либо папку (имя неважно);
- Метод `__len__` должен возвращать количество текстов в папке;
- Метод `__next__`, который позволяет итерироваться по распакованным текстам, должен возвращать объект файла (тот, что возвращает `open()`);
- При итерировании возвращаются нормализованные тексты, т.е. приводятся к нижнему регистру и убираются знаки препинания.

Обратите внимание, что метод `__len__` не должен считывать все файлы. Также нельзя читать все файлы за раз - нужно это делать по мере необходимости.

In [7]:
import zipfile
import os
import re
import pathlib

class TextLoader:
    def __init__(self,url):
        with zipfile.ZipFile('sample.zip','r') as zip_ref:
            zip_ref.extractall(url)
            
        self.path = pathlib.Path('sample')
        self.files = [x for x in list(self.path.glob('**/*')) if x.is_file()]
        self.iterable = iter(self.files)

    def __len__(self):
        return len(self.files)

    def __next__(self):
        file_path = next(self.iterable)
        
        with file_path.open('r', encoding='utf-8') as f:
            read_file = f.read()
            
        with file_path.open('w', encoding='utf-8') as f:
            normalize_file = self.normal(read_file)
            f.write(normalize_file)
        f= file_path.open('r', encoding='utf-8') 
        return f

    def __iter__(self):
        return self

    def normal(self,s):
        s = re.sub(r'[^\w\s]', ' ',s)
        s = s.lower()
        return s

address = '.'
end = TextLoader(address)

print(len(end))

for i in range(len(end)):
    try:
        a = next(end)
        print(a.read()[:100])
        print('----------------------')
    except StopIteration:
        print('finish')

100
  мам  пап  это мой жених сашка 
мама отводит дочку в сторонку 
  ты зачем этого алкаша привела  
  
----------------------
22 	про анфиску

есть у нас в детском саду одна манюня  анфиска  у нас
шкафчики по соседству  ну  шк
----------------------
после новогодних каникул первое  что мы увидели  войдя в раздевалку 
была анфиска  она стояла на сту
----------------------
я вспомнил эту историю вчера  когда забирал шкета из садика 
группа под руководством преподавателя п
----------------------
подходит маленький сын к отцу и спрашивает    пап  скажи  что такое политика    объясню тебе на прим
----------------------
подходит маленький сын к отцу и спрашивает    пап  скажи  что такое политика    объясню тебе на прим
----------------------
мама  папа и сынок едут в машине 
 папа  папа  а вот наша мама водит машину лучше тебя
 хм  ну почем
----------------------
брат приехал из деревни  распродает мебель связи с переездом  рассказал 
вчера приезжали покупатели 
----------------------
28 	

## Coroutines

**3.**(2) От некоторого устройства в режиме реального времени приходят данные. Необходимо написать сопрограмму, которая вычисляет среднее, дисперсию, а также количество элементов в переданном наборе данных с устройства. Результаты работы сопрограмма должна выдавать при отправке соответствующих сигналов.

In [12]:
import numpy as np

class PrintAverage(Exception):
    pass

class PrintDispersion(Exception):
    pass

class PrintQuantity(Exception):
    pass

def coroutines():
    print("Starting coroutine")
    elem = []
    try:
        while True:
            try:
                a = yield
                elem.append(a)
            except PrintDispersion:
                yield np.var(elem)
            except PrintAverage:
                yield np.mean(elem)
            except PrintQuantity:
                yield len(elem)
    finally:
        print("Stop coroutine")

coroutine = coroutines()
next(coroutine)

for i in range(12):
    coroutine.send(i)
    if i%2 == 0:
        print("Average:", coroutine.throw(PrintAverage))
        next(coroutine)
    if i%3 == 0:
        print("Dispersion:", coroutine.throw(PrintDispersion))
        next(coroutine)
    if i%4 == 0:
        print("Quantity:", coroutine.throw(PrintQuantity))
        next(coroutine)
        
coroutine.close()

Starting coroutine
Average: 0.0
Dispersion: 0.0
Quantity: 1
Average: 1.0
Dispersion: 1.25
Average: 2.0
Quantity: 5
Average: 3.0
Dispersion: 4.0
Average: 4.0
Quantity: 9
Dispersion: 8.25
Average: 5.0
Stop coroutine


**4.** (4) Представьте, что у вас настроено взаимодействие с сервером, от которого приходят пакеты, содержащие сообщения от различных клиентов. Обработка каждого из клиентов должна идти в отдельном потоке. 

Реализуйте:

- (**i.**)  Корутину `connect_user`, которая принимает данные авторизации от пользователя, открывает файл с расширением `.txt` и создает на его основе корутину `write_to_file`.
- (**ii.**)   Корутину `write_to_file(f_obj)`, которая записывает переданное планировщиком задач сообщение пользователя в файловый объект, переданный в качестве аргумента при генерации. Также корутина принимает и обрабатывает сигнал об окончании соединения и выходит из сопрограммы.
- (**iii.**)  Планировщик задач, распределяющий задачи по сопроцессам на каждого пользователя.

In [26]:
class Terminate(Exception):
    pass 

class Connect(Exception):
    pass

def connect_user(username):
    with open('message.txt','w', encoding='utf-8') as f:
        yield from write_to_file(f)

def write_to_file(f_obj):
    while True:
        try:
            a = yield
            f_obj.writelines(a)
            f_obj.writelines('\n')
        except Terminate:
            print('finish')
            break
    f_obj.close()
    
def plan():
    user = []
    while True:
        try:
            username = yield
            user.append(username)
        except Connect:
            yield from connect_user(user)
            
 

coroutine = plan()
next(coroutine)
coroutine.throw(Connect)
coroutine.send('hello')
coroutine.send('Hi!')
coroutine.throw(Terminate)
coroutine.close()

finish
